In [21]:
import numpy as np

In [22]:
class ReLU:
    def __init__(self):
        self.mask = None
  
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx


In [23]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        
        return out
        
    def backward(self, dout):

        dx = dout * (1.0-self.out) * self.out
        
        return dx


In [24]:
class Affine:
    def __init__(self, W, b):
        self.b = b
        self.W = W
        self.x = None
        self.dW = None
        self.db = None
        

    def forward(self, x):

        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
        
    def backward(self, dout):
        
        dx = np.dot(dout, self.W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis =0)
        
        return dx

In [25]:
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x)
    exp_all = np.sum(exp_x)
    
    return exp_x / exp_all

def cross_entropy_error(y,t):
    
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

class SoftmaxWithLoss:
    def __init__(self):
        self.y = None
        self.loss = None
        self.t = None
    
    def forward(self, x, t):
        self.y = softmax(x)
        self.t = t
        self.loss = cross_entropy_error(y, t)
        
        return self.loss
        
    
    def backward(self, dout=1):
        
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx
        
    

In [26]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        it.iternext()
        x[idx] = tmp_val
        
    return grad

In [41]:
from collections import OrderedDict
# Ch04에 있는 네트워크와 동일

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['ReLU'] = ReLU()
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        
        for layer in self.layers.values():
            x = layer.forward(x)

        return x
            
      # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y,t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t)/ float(x.shape[0])
        
        return accuracy
        

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        
        # 마지막 레이어 역전파
        dout = self.lastLayer.backward(dout)
        
        # 레이어 뒤집기
        layers = list(self.layers.value())
        layers.reverse()
    
        # 레이어 역전파
        for layer in layers.value:
            dout = layer.backward(dout)

        # 결과 저장
        
        grads = {}
        grads['W1'] = self.layers['W1'].dW
        grads['b1'] = self.layers['b1'].db
        grads['W2'] = self.layers['W2'].dW
        grads['b2'] = self.layers['b2'].db
        
        return grads
        


In [42]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [43]:
x = np.random.rand(100, 784)  # 더미 입력 데이터(100장 분량)
t = np.random.rand(100, 10)   # 더미 정답 레이블(100장 분량)

%time grad_backprop = net.gradient(x, t) # 현재 기울기 계산

ValueError: shapes (100,784) and (100,10) not aligned: 784 (dim 1) != 100 (dim 0)

In [30]:
from dataset03.dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, one_hot_label=True)

In [33]:
# Hyperparameter
iters_num = 1000
batch_size = 100
train_size = x_train.shape[0]
learning_rate = 0.1

net = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []


# 1에폭당 반복 수
iter_per_epoch = max(train_size/batch_size,1)

In [32]:
from tqdm.auto import tqdm # progress bar

for i in tqdm(range(iters_num)):
    # print(i) # Progress Bar를 사용하므로 iter를 출력할 필요 없음

    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grads = net.gradient(x_batch,t_batch)

    # 매개변수 갱신
    for key in ('W1','b1','W2','b2'):
        net.params[key] -= learning_rate * grads[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

   # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "+ str(train_acc) + ", " + str(test_acc))

## 결과가 나와야함!

NameError: name 'iters_num' is not defined